In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../../pre-miRNAs/filtered_premirna_from_S/mirnafold_output_from_S_filtered.csv')
df

,MFE,pre-miRNA candidate seq,MFEIS,class
0,-42.9,CUUUUCUUAUGGACCUUGAAGGAAAACAGGGUAAUUUCAAAAAUCU...,-0.858000,Real
1,-47.9,GCAUCAUUUUCCACUUUUAAGUGUUAUGGAGUGUCUCCUACUAAAU...,-0.903774,Real
2,-27.3,UCAUUUGUAAUUAGAGGUGAUGAAGUCAGACAAAUCGCUCCAGGGC...,-0.880645,Real
3,-25.2,CAGAGUAGUAGUACUUUCUUUUGAACUUCUACAUGCACCAGCAACU...,-0.900000,Pseudo
4,-19.3,AGCAAGUGCACUUGGAAAACUUCAAGAUGUGGUCAACCAAAAUGCA...,-0.877273,Pseudo
5,-28.1,AAAGUAUGAGCAGUAUAUAAAAUGGCCAUGGUACAUUUGGCUAGGU...,-0.851515,Real
6,-25.0,UGCAACGAUACCGAUACAAGCCUCACUCCCUUUCGGAUGGCUUAUU...,-0.862069,Pseudo
7,-43.7,GUAACUUCUUCAAUUGUCAUUACUUCAGGUGAUGGCACAACAAGUC...,-0.891837,Real
8,-30.7,UUGACUUUCAGGUUACUAUAGCAGAGAUAUUACUAAUUAUUAUGAG...,-0.852778,Pseudo
9,-39.4,UCAGACAAGAGGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAU...,-0.985000,Real


In [6]:
filtered_premirnas = df[df['class'] == 'Real']
filtered_premirnas

,MFE,pre-miRNA candidate seq,MFEIS,class,sequence
0,-42.9,CUUUUCUUAUGGACCUUGAAGGAAAACAGGGUAAUUUCAAAAAUCU...,-0.858000,Real,CUUUUCUUAUGGACCUUGAAGGAAAACAGGGUAAUUUCAAAAAUCU...
1,-47.9,GCAUCAUUUUCCACUUUUAAGUGUUAUGGAGUGUCUCCUACUAAAU...,-0.903774,Real,GCAUCAUUUUCCACUUUUAAGUGUUAUGGAGUGUCUCCUACUAAAU...
2,-27.3,UCAUUUGUAAUUAGAGGUGAUGAAGUCAGACAAAUCGCUCCAGGGC...,-0.880645,Real,UCAUUUGUAAUUAGAGGUGAUGAAGUCAGACAAAUCGCUCCAGGGC...
5,-28.1,AAAGUAUGAGCAGUAUAUAAAAUGGCCAUGGUACAUUUGGCUAGGU...,-0.851515,Real,AAAGUAUGAGCAGUAUAUAAAAUGGCCAUGGUACAUUUGGCUAGGU...
7,-43.7,GUAACUUCUUCAAUUGUCAUUACUUCAGGUGAUGGCACAACAAGUC...,-0.891837,Real,GUAACUUCUUCAAUUGUCAUUACUUCAGGUGAUGGCACAACAAGUC...
9,-39.4,UCAGACAAGAGGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAU...,-0.985000,Real,UCAGACAAGAGGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAU...
10,-36.7,GGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAUUGUUGCGGCA...,-0.965789,Real,GGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAUUGUUGCGGCA...


In [9]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=7139ff5c07327e5dd4deb2418e2dcb8cd3b31419fb225d16e64779f76f85845f
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [12]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [13]:
import re

def parse_bayes_output(soup_html):
    stem_re = "\d' *stem"
    rna_re = "[AUGC]+"

    seqs = [el.text for el in html_soup.select('tr')[2:]]
    seqs = [(re.findall(rna_re, mirna), re.findall(stem_re, mirna)) for mirna in seqs]
    seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
#     output = []
#     for strand_match, sequence_match in seqs:
#         strand = strand_match[0]
#         sequence = sequence_match[0]
#         print(sequence)
#         if len(strand) > 0 and len(sequence) > 0:
#             output.append((strand, sequence))
#         seqs = [(strand[0], sequence[0][:2]) for strand, sequence in seqs if len(strand) > 0 and len(sequence) > 0]
    return list(set(seqs))

In [14]:
mature_bayes_url = "http://mirna.imbb.forth.gr/cgi-bin/MatureBayes/maturebayes.py"
mature_mirnas = []

for i in tqdm(range(df.shape[0])):
    premirna_seq = df.iloc[i]['sequence']
    payload = {"single_seq": premirna_seq}
    
    response = requests.post(mature_bayes_url, payload)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    new_mature_mirnas = parse_bayes_output(html_soup)
    mature_mirnas += new_mature_mirnas
    
mature_mirnas = list(set(mature_mirnas))
print(f'Found {len(mature_mirnas)} mature miRNA candidates.')

  0%|          | 0/11 [00:00<?, ?it/s]

Found 31 mature miRNA candidates.


In [15]:
mature_mirnas = pd.DataFrame(mature_mirnas, columns=['sequence', 'strand'])
mature_mirnas

,sequence,strand
0,AGUUUCCAUUUGGAAUCUUGAU,5'
1,AAGUUCAAGAACUUUACUCUCC,5'
2,AUGAAGUCAGACAAAUCGCUCC,5'
3,UACAUGCACCAGCAACUGUUUG,5'
4,AUAGUGUUUAUAACACUUUGCU,5'
5,AAAGACAGAAUGAUUGAACUUU,3'
6,UAUGAGCAGUAUAUAAAAUGGC,5'
7,UGUGGACCUAAAAAGUCUACUA,3'
8,AUUCAUUUGUAAUUAGAGGUGA,3'
9,UAUUCUAAGCACACGCCUAUUA,3'


In [16]:
mature_mirnas.to_csv('mature-mirnas-from-mirnafold.csv', index=False)